In [2]:
import pandas as pd

covid = pd.read_csv('/work/SQL-Journey/CovidDeaths.csv')
vaccines = pd.read_csv('/work/Vaccines.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/work/SQL-Journey/CovidDeaths.csv'

In [1]:
SELECT *
FROM covid
LIMIT 5

SyntaxError: invalid syntax (3651336915.py, line 1)

In [ ]:
SELECT *
FROM covid
WHERE continent IS NOT null 
ORDER BY 3,4;

Let's select the data we are going to be starting with

In [ ]:
SELECT location, date, total_cases, new_cases, total_deaths, population
FROM covid
    Where continent IS NOT null 
        ORDER BY 1,2

Here we will make a comparasion of Total cases vs Total Deaths, the idea is to show the likelihood of dying if you get covid in your country, let's use 'Colombia' as an example.

In [ ]:
SELECT location, date, total_cases,total_deaths, ROUND((total_deaths/total_cases)*100,2) AS DeathPercentage
FROM covid
WHERE location = 'Colombia'
      AND continent IS NOT null 
ORDER BY 2 DESC;

Now let's make the same exercise but with population, Cases Vs Population

In [ ]:
SELECT location, date, Population, total_cases,  ROUND((total_cases/population)*100,2) as Percent_PopulationInfected
FROM covid
    WHERE location = 'Colombia'
        ORDER BY 2 DESC;

In [ ]:
SELECT location, date, Population, total_cases,  ROUND((total_cases/population)*100,2) as Percent_PopulationInfected
FROM covid
    WHERE location = 'Colombia'
        ORDER BY 2 DESC;

Countries with Highest Infection Rate compared to Population

In [ ]:
SELECT  location, 
        Population, 
        MAX(total_cases) as HighestInfectionCount,  
        ROUND(MAX((total_cases/population))*100,2) AS Percent_PopulationInfected

FROM covid
    GROUP BY Location, Population
        HAVING Percent_PopulationInfected != 'nan'
ORDER BY Percent_PopulationInfected DESC;

Now i want to know the countries with the highest death count per population

In [ ]:
SELECT location, MAX(cast(Total_deaths as int)) AS TotalDeathCount
From covid

WHERE continent IS NOT null  
    GROUP BY location
        ORDER BY TotalDeathCount DESC
        OFFSET 28;  #if you dont want to exclude the values you can skip them with this.

Now let's do the same thing but i want to break it down by continent

In [ ]:
SELECT continent, MAX(cast(Total_deaths as int)) AS TotalDeathCount
FROM covid

WHERE continent IS NOT null  
    GROUP BY continent
        ORDER BY TotalDeathCount DESC;

-- Yeah yeah...i know, i'm also bothered that this dataset states that america has 2 continents.


Let's compare the Total Population vs Vaccinations
We want to show the Percentage of Population that has recieved at least one Covid Vaccine

In [ ]:
SELECT a.continent, 
        a.location, 
        a.date, 
        a.population, 
        vac.new_Vaccinations,
        SUM(vac.new_vaccinations) OVER (Partition by a.Location ORDER BY a.location, a.date) AS RollingPeopleVaccinated,
        --(RollingPeopleVaccinated/population)*100 AS percentig
FROM covid AS a
    JOIN vaccines AS vac ON a.location = vac.location
	AND a.date = vac.date

WHERE a.continent IS NOT null 
ORDER BY 2,3, RollingPeopleVaccinated DESC;


Here we use a CTE to perform a calculation from the previous query

In [ ]:


df_1 = _deepnote_execute_sql("""
With PopvsVac (Continent, Location, Date, Population, New_Vaccinations, RollingPeopleVaccinated)
as
(
Select a.continent, a.location, a.date, a.population, vac.new_vaccinations
, SUM(CONVERT(int,vac.new_vaccinations)) OVER (Partition by a.Location Order by a.location, a.Date) as RollingPeopleVaccinated
--, (RollingPeopleVaccinated/population)*100
From covid AS a
Join vaccines AS vac
	On a.location = vac.location
	and a.date = vac.date
where a.continent is not null 
--order by 2,3
)
Select *, (RollingPeopleVaccinated/Population)*100
From PopvsVac
""", 'SQL_DEEPNOTE_DATAFRAME_SQL')
df_1

Now let´s just create a view to store data for later visualizations


In [ ]:


df_2 = _deepnote_execute_sql("""Create View PercentPopulationVaccinated as
Select dea.continent, dea.location, dea.date, dea.population, vac.new_vaccinations
, SUM(CONVERT(int,vac.new_vaccinations)) OVER (Partition by dea.Location Order by dea.location, dea.Date) as RollingPeopleVaccinated
--, (RollingPeopleVaccinated/population)*100
From covid AS dea
Join Vaccines AS vac
	On dea.location = vac.location
	and dea.date = vac.date
where dea.continent is not null 
""", 'SQL_DEEPNOTE_DATAFRAME_SQL')
df_2

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=b390b82e-1599-4337-8017-92f9efdb1ebc' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>